In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


As you may see in the PLINK log file (`/scratch/groups/mrivas/ukbb24983/array-exome-combined/pgen/ukb24983_cal_hla_cnv_exomeOQFE.bed.log`), 
there are some variants removed based on the allele frequency threshold.

> 2962 variants removed due to minor allele threshold(s)

We add back the "geno_data_source: column.


```
$ cd /scratch/groups/mrivas/ukbb24983/array-exome-combined/pgen

$ cat ukb24983_cal_hla_cnv_exomeOQFE.pvar | md5sum
7ac090a4b9bc4686cd747132f7b62785  -

$ mv ukb24983_cal_hla_cnv_exomeOQFE.pvar ukb24983_cal_hla_cnv_exomeOQFE.plink.pvar

$ we execute the codes in this notebook and generate the merged file

$ cat ukb24983_cal_hla_cnv_exomeOQFE.pvar| cut -f1-5 | md5sum
7ac090a4b9bc4686cd747132f7b62785  -
```


In [2]:
data_d <- '/scratch/groups/mrivas/ukbb24983/array-exome-combined/pgen'
merge_list <- file.path(data_d, 'merge_list_pvar', 'ukb24983_cal_hla_cnv_exomeOQFE.pvar.gz')
in_f <- file.path(data_d, 'ukb24983_cal_hla_cnv_exomeOQFE.plink.pvar')
out_f <- file.path(data_d, 'ukb24983_cal_hla_cnv_exomeOQFE.pvar')


In [3]:
merge_list %>%
fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> merge_list_df


In [5]:
in_f %>%
fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> df


In [7]:
df %>%
left_join(
    merge_list_df %>% select(ID, geno_data_source), by='ID'
) -> merged_df


In [9]:
merged_df %>% count(geno_data_source)

geno_data_source,n
<chr>,<int>
cal,802498
cnv,275180
exome200k,17660693
hla,328


In [11]:
merged_df %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)
